### Agentic Chain

In [ ]:
# Step 1: Load, Chunk, Embed, Store

from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_ollama import OllamaEmbeddings

# Load documents from a URL
docs = WebBaseLoader("https://docs.smith.langchain.com/user_guide").load()  

# Split documents into chunks
chunk = RecursiveCharacterTextSplitter()
documents = chunk.split_documents(docs)              
document_texts = [doc.page_content for doc in documents]  # Extract text

# Embed document texts
embed = OllamaEmbeddings(model="nomic-embed-text")
embeddings = embed.embed_documents(document_texts)  # Generate embeddings

# Create and save FAISS vector store
vector = FAISS.from_embeddings(list(zip(document_texts, embeddings)), embedding=embed)
vector.save_local("C:/Users/admin/OneDrive/Desktop/RAG'S/vector db/Faiss-index/")  # Store locally

In [ ]:
import os
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_groq import ChatGroq
from langchain_community.vectorstores import FAISS
from langchain_ollama import OllamaEmbeddings
from langchain.tools.retriever import create_retriever_tool
from langchain_community.tools import DuckDuckGoSearchResults  # DuckDuckGo for general search
from langchain import hub
from langchain.agents import AgentExecutor, create_tool_calling_agent

# Load vector Store
embed = OllamaEmbeddings(model="nomic-embed-text")
vector = FAISS.load_local("C:/Users/admin/OneDrive/Desktop/RAG'S/vector db/Faiss-index/", embeddings=embed, allow_dangerous_deserialization=True)

#Retriever tool
retriever = vector.as_retriever()
retriever_tool = create_retriever_tool(
    retriever,
    "langsmith_search",
    "Search for information about LangSmith. For any questions about LangSmith, you must use this tool!"
)

# Search tool
os.environ["TAVILY_API_KEY"] = "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"
search_tool = TavilySearchResults()

tools = [retriever_tool, search_tool]

prompt = hub.pull("hwchase17/openai-functions-agent")

os.environ["GROQ_API_KEY"] = "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"
llm_mixtral = ChatGroq(model="mixtral-8x7b-32768")

# Create agent using the Groq LLM and tools
agent = create_tool_calling_agent(tools, prompt, llm_mixtral)

# Initialize AgentExecutor to run the agent with tools
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

c:\Users\admin\anaconda3\envs\venv\Lib\site-packages\langsmith\client.py:256: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [24]:
response = agent_executor.invoke({"input": "What is LangSmith in short?"})
print(response)




> Entering new AgentExecutor chain...

Invoking: `langsmith_search` with `{'query': 'LangSmith overview'}`


Beta Testing​
Beta testing allows developers to collect more data on how their LLM applications are performing in real-world scenarios. In this phase, it’s important to develop an understanding for the types of inputs the app is performing well or poorly on and how exactly it’s breaking down in those cases. Both feedback collection and run annotation are critical for this workflow. This will help in curation of test cases that can help track regressions/improvements and development of automatic evaluations.
Capturing Feedback​
When launching your application to an initial set of users, it’s important to gather human feedback on the responses it’s producing. This helps draw attention to the most interesting runs and highlight edge cases that are causing problematic responses. LangSmith allows you to attach feedback scores to logged traces (oftentimes, this is hooked up to a fee

In [25]:
print(response['output'])

LangSmith is a platform for large language model (LLM) application development, monitoring, and testing. During the prototyping phase, it is helpful to have LangSmith tracing enabled, allowing developers to quickly understand how the model is performing and debug any issues. LangSmith also allows developers to create datasets of inputs and reference outputs, and use these to run tests on their LLM applications. The comparison view allows developers to see how different configurations of their application perform side-by-side. LangSmith also offers a playground for rapid iteration and experimentation.

In the beta testing phase, LangSmith allows developers to gather human feedback on the responses their application is producing and attach feedback scores to logged traces. This helps highlight edge cases that are causing problematic responses and draw attention to the most interesting runs. LangSmith also supports sending runs to annotation queues, where annotators can closely inspect an